In [ ]:
import random
import heapq
from typing import List, Tuple, Optional
from PIL import Image, ImageDraw
import numpy as np


def _odd(n: int) -> int:
    return n if n % 2 == 1 else n - 1


def generate_maze_grid(
        rows: int, 
        cols: int, 
        seed: Optional[int] = None
    ) -> List[List[int]]:
    """
    create a binary grid maze sized rows x cols.
    1=open path, 0=wall. Uses randomized DFS on an odd-sized logical grid.
    if rows/cols are even, they’re reduced by 1 to stay odd (to keep proper corridors).
    """
    if seed is not None:
        random.seed(seed)
    R = max(5, _odd(rows))
    C = max(5, _odd(cols))
    maze = np.zeros((R, C), dtype=np.uint8)
    def carve():
        start_r = random.randrange(1, R, 2)
        start_c = random.randrange(1, C, 2)
        stack = [(start_r, start_c)]
        maze[start_r, start_c] = 1
        dirs = [(-2,0), (2,0), (0,-2), (0,2)]

        while stack:
            r, c = stack[-1]
            random.shuffle(dirs)
            moved = False
            for dr, dc in dirs:
                nr, nc = r + dr, c + dc
                if 1 <= nr < R-1 and 1 <= nc < C-1 and maze[nr, nc] == 0:
                    maze[nr, nc] = 1
                    maze[r + dr//2, c + dc//2] = 1
                    stack.append((nr, nc))
                    moved = True
                    break
            if not moved:
                stack.pop()
    carve()
    tops = [c for c in range(1, C-1, 2) if maze[1, c] == 1]
    bots = [c for c in range(1, C-1, 2) if maze[R-2, c] == 1]
    if tops:
        c = random.choice(tops)
        maze[0, c] = 1
    if bots:
        c = random.choice(bots)
        maze[R-1, c] = 1
    return maze.tolist()


class Cell:
    def __init__(self):
        self.parent_i = 0
        self.parent_j = 0
        self.f = float('inf')
        self.g = float('inf')
        self.h = 0.0


def is_valid(grid: List[List[int]], row: int, col: int) -> bool:
    return 0 <= row < len(grid) and 0 <= col < len(grid[0])


def is_unblocked(grid: List[List[int]], row: int, col: int) -> bool:
    return grid[row][col] == 1


def is_destination(row: int, col: int, dest: Tuple[int,int]) -> bool:
    return row == dest[0] and col == dest[1]


def trace_path(cell_details: List[List[Cell]], dest: Tuple[int,int]) -> List[Tuple[int,int]]:
    path = []
    row, col = dest
    while not (cell_details[row][col].parent_i == row and cell_details[row][col].parent_j == col):
        path.append((row, col))
        tr = cell_details[row][col].parent_i
        tc = cell_details[row][col].parent_j
        row, col = tr, tc
    path.append((row, col))
    path.reverse()
    return path


def grid_to_image(grid: List[List[int]], pixels_per_cell: int = 1) -> Image.Image:
    """render a 1/0 grid to a black/white PIL image"""
    arr = (np.array(grid, dtype=np.uint8) * 255)
    img = Image.fromarray(arr, mode="L")
    if pixels_per_cell != 1:
        img = img.resize(
            (img.width * pixels_per_cell, img.height * pixels_per_cell),
            resample=Image.NEAREST
        )
    return img


def overlay_path(img: Image.Image, path: List[Tuple[int,int]], pixels_per_cell: int = 1,
                 color=(255,0,0), thickness: int = 1) -> Image.Image:
    """overlay the solution path on a copy of the maze image"""
    if img.mode != "RGB":
        base = img.convert("RGB")
    else:
        base = img.copy()
    draw = ImageDraw.Draw(base)
    for (r, c) in path:
        x0 = c * pixels_per_cell
        y0 = r * pixels_per_cell
        x1 = x0 + pixels_per_cell - 1
        y1 = y0 + pixels_per_cell - 1
        for t in range(thickness):
            draw.rectangle([x0+t, y0+t, x1-t, y1-t], fill=color)
    return base


# TODO

A-star selects the next node with the lowest estimated cost 'f' value.  
The f value calculated by adding the g and h values.

A node's g value is the cost of reaching that node from the start.  
In an image this is simply the previous node's distance plus 1.

A node's h value is the estimated cost of reaching the node from the destination.  
In a grid, this is a simple euclidean distance calculation.

Complete the calculations for f, g, and h:

In [ ]:
def calculate_h_value(row: int, col: int, dest: Tuple[int,int]) -> float:
    h = # TODO: calculate h
    return h


def a_star_search(
        grid: List[List[int]], 
        src: Tuple[int,int], 
        dest: Tuple[int,int]
    ) -> Optional[List[Tuple[int,int]]]:
    """a star path planning algorithm"""
    if not is_valid(grid, src[0], src[1]) or not is_valid(grid, dest[0], dest[1]):
        print("Source or destination is invalid")
        return None
    if not is_unblocked(grid, src[0], src[1]) or not is_unblocked(grid, dest[0], dest[1]):
        print("Source or the destination is blocked")
        return None
    if is_destination(src[0], src[1], dest):
        return [src]

    num_rows, num_cols = len(grid), len(grid[0])
    closed_list = [[False]*num_cols for _ in range(num_rows)]
    cells = [[Cell() for _ in range(num_cols)] for _ in range(num_rows)]

    i, j = src
    cells[i][j].f = cells[i][j].g = cells[i][j].h = 0.0
    cells[i][j].parent_i = i
    cells[i][j].parent_j = j

    open_list = []
    heapq.heappush(open_list, (0.0, i, j))

    directions = [(0,1),(0,-1),(1,0),(-1,0)] # only move orthogonally

    while open_list:
        _, i, j = heapq.heappop(open_list) # heapq always keeps the smallest object as the first object
        closed_list[i][j] = True

        for di, dj in directions:
            ni, nj = i + di, j + dj
            if is_valid(grid, ni, nj) and is_unblocked(grid, ni, nj) and not closed_list[ni][nj]:
                if is_destination(ni, nj, dest):
                    cells[ni][nj].parent_i = i
                    cells[ni][nj].parent_j = j
                    return trace_path(cells, dest)

                g_new = # TODO: calculate the new g value
                h_new = calculate_h_value(ni, nj, dest)
                f_new = # TODO: calculate the new f value

                if cells[ni][nj].f == float('inf') or cells[ni][nj].f > f_new:
                    heapq.heappush(open_list, (f_new, ni, nj))
                    cells[ni][nj].f = f_new
                    cells[ni][nj].g = g_new
                    cells[ni][nj].h = h_new
                    cells[ni][nj].parent_i = i
                    cells[ni][nj].parent_j = j

    print("Failed to find the destination cell")
    return None

In [ ]:
def generate_and_solve(seed=0):
    rows, cols = 51, 51
    grid = generate_maze_grid(rows, cols, seed=seed)
    top_open = [(0, c) for c in range(cols) if grid[0][c] == 1]
    bot_open = [(rows-1, c) for c in range(cols) if grid[rows-1][c] == 1]
    if not top_open:
        top_open = [(0, 1)]
    if not bot_open:
        bot_open = [(rows-1, cols-2)]
    src = random.choice(bot_open)
    dest = random.choice(top_open)

    path = a_star_search(grid, src, dest)
    maze_img = grid_to_image(grid, pixels_per_cell=4)
    display(maze_img)

    if path:
        solved = overlay_path(maze_img, path, pixels_per_cell=4, thickness=1)
        display(solved)
        print(f"Path length: {len(path)} cells")
        print(f"Src: {src}  Dest: {dest}")
    else:
        print("No path found (unexpected for perfect mazes).")

In [ ]:
generate_and_solve(seed=0)